In [12]:
''' summarize a webpage to use an Open Source model running locally via Ollama downloaded package and 
usingrather than OpenAI comment
'''
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [20]:
# Constants

MODEL = "deepseek-r1:1.5b"

In [21]:
class Website:
    url: str
    title:str
    text:str
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input", "think"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [22]:
# system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


# user prompt

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
\nPlease provide a short summary of this website in markdown. \
\nIf it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [23]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [24]:
def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [25]:
summarize("https://edwarddonner.com")

'<think>\nOkay, so I need to summarize this website. Let\'s see what it has.\n\nFirst, the main sections are Home, Connect Four, Outsmart, and An arena that pits LLMs against each other in a battle of diplomacy and deviousness. These seem like features or platforms. The About section lists some topics like posts, code writing, music production, etc., so maybe it\'s an online community for developers.\n\nThere are links to Connect Four, Outsmart, and another arena, which sounds like a game or platform. The About also mentions "Get in touch" with contact info, website, and social media. There\'s also a newsletter subscription section, which is useful for staying updated on stuff.\n\nThe structure seems similar to some other online communities, but I need to make it concise without diving into details. So the summary should mention multiple platforms related to AI applications, developer support, and community engagement.\n\nWait, does "Outsmart" have anything specific? Maybe something li